Simulator

In [13]:
import numpy as np
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider
output_notebook()

Loading BokehJS ...

In [14]:
# ground airplane speed
v_0 = 148.16 * 1e3 / 3600 # 80 kt converted in m/s
# ground wind speed
v_w = 0
# gravitational acceleration
g = 9.81 # m.s^-2
# drop altitude
y_0 = 4000 # m (with respect to the gound)
# skydiver mass
m = 70 # kg 
# air volumic mass
rho = 1 # kg.m^3 env. 1.292 at sea level and 0.99 at 2200 m

In [15]:
#C_x = 1.2 # homme debout
#S_x = C_x * 0.42 # m^2 (surface initiale du parachutiste selon Ox)

In [16]:
def x_drag(t, m, rho, C_x, S_x, v_0, v_w):
    t = np.array(t)
    S_x = C_x * S_x
    a = 2*m/rho/S_x
    v_x0 = v_0 + v_w
    return a*np.log((rho*S_x*v_x0*t + 2*m)/(2*m))

def y_drag(t, m, rho, C_y, S_y):
    global g
    t = np.array(t)
    C_y = 1 # (human body)
    S_y = C_y*0.42 # m^2 (surface parachutiste parachute non déployé)
    v_lim = (2*m*g/rho/S_y)**0.5
    tau = v_lim / g
    return - v_lim * t - v_lim * tau * np.log((1+np.exp(-2*t/tau))/2) + y_0

In [17]:
C_x = 1.2
S_x = 0.42
C_y = 1
S_y = 0.42

t=np.arange(0,30)

y = y_drag(t, m, rho, C_y, S_y)
x = x_drag(t, m, rho, C_x, S_x, v_0, v_w)

In [18]:
source = ColumnDataSource(data=dict(x=x, y=x))

plot = figure(plot_width=400, plot_height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

slider = Slider(start=0.1, end=6, value=1, step=.1, title="power")

update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
    var data = source.data;
    var f = slider.value;
    var x = data['x']
    var y = data['y']
    for (var i = 0; i < x.length; i++) {
        y[i] = Math.pow(x[i], f)
    }
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)


show(column(slider, plot))